<a href="https://colab.research.google.com/github/codingniket/Python-Training/blob/main/22_12_2025/Exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
spark = SparkSession.builder.appName("SparkByExamples.com")\
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0")\
    .getOrCreate()

In [ ]:
raw_orders = [
    ("ORD001","C001","Ravi"," Delhi ","Laptop","Electronics","45000","2024-01-05","Completed"),
    ("ORD002","C002","Sneha","Mumbai"," Mobile ","Electronics","32000","05/01/2024","Completed"),
    ("ORD003","C003","Aman","Bangalore","Laptop","Electronics","55000","2024/01/06","Completed"),
    ("ORD004","C004","Pooja","Delhi","Tablet"," Electronics ","","2024-01-07","Cancelled"),
    ("ORD005","C005","Neha","Chennai","Laptop","Electronics","48000","invalid_date","Completed"),
    ("ORD006","C006","Rahul","Mumbai","Mobile","Electronics",None,"2024-01-08","Completed"),
    ("ORD007","C007","Kiran","Bangalore","Tablet","Electronics","30000","2024-01-08","Completed"),
    ("ORD008","C008","Amit","Delhi","Laptop","electronics","45000","2024-01-09","Completed"),
    ("ORD009","C009","Priya"," Pune","Mobile","Electronics","28000","09-01-2024","Completed"),
    ("ORD010","C010","Suresh","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed"),
    ("ORD010","C010","Suresh","Mumbai","Laptop","Electronics","55000","2024-01-10","Completed"),  # duplicate
    ("ORD011","C011","Meena","Chennai","Tablet","Electronics","31000","2024-01-11","Completed"),
    ("ORD012","C012","Arjun","Delhi","Mobile","Electronics","27000","2024/01/11","Completed"),
    ("ORD013","C013","Nikhil","Bangalore","Laptop","Electronics","60000","2024-01-12","Completed"),
    ("ORD014","C014","Rohit","Mumbai","Mobile","Electronics","invalid_price","2024-01-12","Completed"),
    ("ORD015","C015","Anita","Delhi","Tablet","Electronics","29000","2024-01-13","Completed"),
    ("ORD016","C016","Vikas","Chennai","Laptop","Electronics","52000","2024-01-13","Completed"),
    ("ORD017","C017","Sunita","Mumbai","Mobile","Electronics","33000","2024-01-14","Completed"),
    ("ORD018","C018","Deepak","Bangalore","Laptop","Electronics","58000","2024-01-14","Completed"),
    ("ORD019","C019","Pallavi","Delhi","Mobile","Electronics","26000","2024-01-15","Completed"),
    ("ORD020","C020","Manish","Mumbai","Tablet","Electronics","34000","2024-01-15","Completed")
]


In [ ]:
columns = ["orderId","customerId","customerName","city","product","category","price","orderDate","orderStatus"]

In [ ]:
df = spark.createDataFrame(raw_orders,columns)
df.show()

+-------+----------+------------+---------+--------+-------------+-------------+------------+-----------+
|orderId|customerId|customerName|     city| product|     category|        price|   orderDate|orderStatus|
+-------+----------+------------+---------+--------+-------------+-------------+------------+-----------+
| ORD001|      C001|        Ravi|   Delhi |  Laptop|  Electronics|        45000|  2024-01-05|  Completed|
| ORD002|      C002|       Sneha|   Mumbai| Mobile |  Electronics|        32000|  05/01/2024|  Completed|
| ORD003|      C003|        Aman|Bangalore|  Laptop|  Electronics|        55000|  2024/01/06|  Completed|
| ORD004|      C004|       Pooja|    Delhi|  Tablet| Electronics |             |  2024-01-07|  Cancelled|
| ORD005|      C005|        Neha|  Chennai|  Laptop|  Electronics|        48000|invalid_date|  Completed|
| ORD006|      C006|       Rahul|   Mumbai|  Mobile|  Electronics|         NULL|  2024-01-08|  Completed|
| ORD007|      C007|       Kiran|Bangalore|  T

Fixing Dates Here 👇

In [5]:
from pyspark.sql.functions import coalesce, to_date, try_to_timestamp

In [6]:
fixed_date =df.withColumn("orderDate",
                                    coalesce(
                                        to_date(try_to_timestamp(col("orderDate"),lit("yyyy-MM-dd"))),
                                        to_date(try_to_timestamp(col("orderDate"),lit("dd/MM/yyyy"))),
                                        to_date(try_to_timestamp(col("orderDate"),lit("yyyy/MM/dd")))
                                    )
                                   )
fixed_date.show()

+-------+----------+------------+---------+--------+-------------+-------------+----------+-----------+
|orderId|customerId|customerName|     city| product|     category|        price| orderDate|orderStatus|
+-------+----------+------------+---------+--------+-------------+-------------+----------+-----------+
| ORD001|      C001|        Ravi|   Delhi |  Laptop|  Electronics|        45000|2024-01-05|  Completed|
| ORD002|      C002|       Sneha|   Mumbai| Mobile |  Electronics|        32000|2024-01-05|  Completed|
| ORD003|      C003|        Aman|Bangalore|  Laptop|  Electronics|        55000|2024-01-06|  Completed|
| ORD004|      C004|       Pooja|    Delhi|  Tablet| Electronics |             |2024-01-07|  Cancelled|
| ORD005|      C005|        Neha|  Chennai|  Laptop|  Electronics|        48000|      NULL|  Completed|
| ORD006|      C006|       Rahul|   Mumbai|  Mobile|  Electronics|         NULL|2024-01-08|  Completed|
| ORD007|      C007|       Kiran|Bangalore|  Tablet|  Electronic

Fixing category Here 👇

In [8]:
from pyspark.sql.functions import col, trim, lower

In [11]:
fixed_category = fixed_date.withColumn("category",lower(trim(col("category"))))
fixed_category.show()

+-------+----------+------------+---------+--------+-----------+-------------+----------+-----------+
|orderId|customerId|customerName|     city| product|   category|        price| orderDate|orderStatus|
+-------+----------+------------+---------+--------+-----------+-------------+----------+-----------+
| ORD001|      C001|        Ravi|   Delhi |  Laptop|electronics|        45000|2024-01-05|  Completed|
| ORD002|      C002|       Sneha|   Mumbai| Mobile |electronics|        32000|2024-01-05|  Completed|
| ORD003|      C003|        Aman|Bangalore|  Laptop|electronics|        55000|2024-01-06|  Completed|
| ORD004|      C004|       Pooja|    Delhi|  Tablet|electronics|             |2024-01-07|  Cancelled|
| ORD005|      C005|        Neha|  Chennai|  Laptop|electronics|        48000|      NULL|  Completed|
| ORD006|      C006|       Rahul|   Mumbai|  Mobile|electronics|         NULL|2024-01-08|  Completed|
| ORD007|      C007|       Kiran|Bangalore|  Tablet|electronics|        30000|2024

Fixing city & product Here 👇

In [13]:
fixed_city_product = fixed_category.withColumn("city",trim(col("city")))\
.withColumn("product",trim(col("product")))
fixed_city_product.show()

+-------+----------+------------+---------+-------+-----------+-------------+----------+-----------+
|orderId|customerId|customerName|     city|product|   category|        price| orderDate|orderStatus|
+-------+----------+------------+---------+-------+-----------+-------------+----------+-----------+
| ORD001|      C001|        Ravi|    Delhi| Laptop|electronics|        45000|2024-01-05|  Completed|
| ORD002|      C002|       Sneha|   Mumbai| Mobile|electronics|        32000|2024-01-05|  Completed|
| ORD003|      C003|        Aman|Bangalore| Laptop|electronics|        55000|2024-01-06|  Completed|
| ORD004|      C004|       Pooja|    Delhi| Tablet|electronics|             |2024-01-07|  Cancelled|
| ORD005|      C005|        Neha|  Chennai| Laptop|electronics|        48000|      NULL|  Completed|
| ORD006|      C006|       Rahul|   Mumbai| Mobile|electronics|         NULL|2024-01-08|  Completed|
| ORD007|      C007|       Kiran|Bangalore| Tablet|electronics|        30000|2024-01-08|  C

Prices fixed Here 👇

In [19]:
from pyspark.sql.functions import when

In [26]:
from pyspark.sql.functions import col, lit, when, regexp_extract

numeric_price_str = regexp_extract(col("price"), "(\\d+)", 0)

fixed_price = fixed_city_product.withColumn("price",
                                            when((numeric_price_str == "") | numeric_price_str.isNull(), lit(0))
                                           .otherwise(numeric_price_str.cast('int'))
                                           )
fixed_price.show()

+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+
|orderId|customerId|customerName|     city|product|   category|price| orderDate|orderStatus|
+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+
| ORD001|      C001|        Ravi|    Delhi| Laptop|electronics|45000|2024-01-05|  Completed|
| ORD002|      C002|       Sneha|   Mumbai| Mobile|electronics|32000|2024-01-05|  Completed|
| ORD003|      C003|        Aman|Bangalore| Laptop|electronics|55000|2024-01-06|  Completed|
| ORD004|      C004|       Pooja|    Delhi| Tablet|electronics|    0|2024-01-07|  Cancelled|
| ORD005|      C005|        Neha|  Chennai| Laptop|electronics|48000|      NULL|  Completed|
| ORD006|      C006|       Rahul|   Mumbai| Mobile|electronics|    0|2024-01-08|  Completed|
| ORD007|      C007|       Kiran|Bangalore| Tablet|electronics|30000|2024-01-08|  Completed|
| ORD008|      C008|        Amit|    Delhi| Laptop|electronics|45000|2

In [30]:
fixed_price.printSchema()

root
 |-- orderId: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- customerName: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- orderDate: date (nullable = true)
 |-- orderStatus: string (nullable = true)



Removing Duplicate record here 👇

In [35]:
unique_orders = fixed_price.dropDuplicates(['orderId'])
unique_orders.show()

+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+
|orderId|customerId|customerName|     city|product|   category|price| orderDate|orderStatus|
+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+
| ORD001|      C001|        Ravi|    Delhi| Laptop|electronics|45000|2024-01-05|  Completed|
| ORD002|      C002|       Sneha|   Mumbai| Mobile|electronics|32000|2024-01-05|  Completed|
| ORD003|      C003|        Aman|Bangalore| Laptop|electronics|55000|2024-01-06|  Completed|
| ORD004|      C004|       Pooja|    Delhi| Tablet|electronics|    0|2024-01-07|  Cancelled|
| ORD005|      C005|        Neha|  Chennai| Laptop|electronics|48000|      NULL|  Completed|
| ORD006|      C006|       Rahul|   Mumbai| Mobile|electronics|    0|2024-01-08|  Completed|
| ORD007|      C007|       Kiran|Bangalore| Tablet|electronics|30000|2024-01-08|  Completed|
| ORD008|      C008|        Amit|    Delhi| Laptop|electronics|45000|2

Removing not Order

In [38]:
null_date = unique_orders.filter(~col("orderDate").isNull())
null_date.show()

+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+
|orderId|customerId|customerName|     city|product|   category|price| orderDate|orderStatus|
+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+
| ORD001|      C001|        Ravi|    Delhi| Laptop|electronics|45000|2024-01-05|  Completed|
| ORD002|      C002|       Sneha|   Mumbai| Mobile|electronics|32000|2024-01-05|  Completed|
| ORD003|      C003|        Aman|Bangalore| Laptop|electronics|55000|2024-01-06|  Completed|
| ORD004|      C004|       Pooja|    Delhi| Tablet|electronics|    0|2024-01-07|  Cancelled|
| ORD006|      C006|       Rahul|   Mumbai| Mobile|electronics|    0|2024-01-08|  Completed|
| ORD007|      C007|       Kiran|Bangalore| Tablet|electronics|30000|2024-01-08|  Completed|
| ORD008|      C008|        Amit|    Delhi| Laptop|electronics|45000|2024-01-09|  Completed|
| ORD010|      C010|      Suresh|   Mumbai| Laptop|electronics|55000|2

1

In [40]:
price_with_tax = null_date.withColumn("price_with_tax",col("price")+col("price")*0.18)
price_with_tax.show()


+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+
|orderId|customerId|customerName|     city|product|   category|price| orderDate|orderStatus|price_with_tax|
+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+
| ORD001|      C001|        Ravi|    Delhi| Laptop|electronics|45000|2024-01-05|  Completed|       53100.0|
| ORD002|      C002|       Sneha|   Mumbai| Mobile|electronics|32000|2024-01-05|  Completed|       37760.0|
| ORD003|      C003|        Aman|Bangalore| Laptop|electronics|55000|2024-01-06|  Completed|       64900.0|
| ORD004|      C004|       Pooja|    Delhi| Tablet|electronics|    0|2024-01-07|  Cancelled|           0.0|
| ORD006|      C006|       Rahul|   Mumbai| Mobile|electronics|    0|2024-01-08|  Completed|           0.0|
| ORD007|      C007|       Kiran|Bangalore| Tablet|electronics|30000|2024-01-08|  Completed|       35400.0|
| ORD008|      C008|        

In [42]:
price_category = price_with_tax.withColumn("price_category",
                                            when(col("price_with_tax") > 30000,lit("High"))
                                           .when((col("price_with_tax") > 10000) & (col("price_with_tax") <= 30000),lit("Medium"))
                                           .otherwise(lit("Low"))
                                           )
price_category.show()

+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+--------------+
|orderId|customerId|customerName|     city|product|   category|price| orderDate|orderStatus|price_with_tax|price_category|
+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+--------------+
| ORD001|      C001|        Ravi|    Delhi| Laptop|electronics|45000|2024-01-05|  Completed|       53100.0|          High|
| ORD002|      C002|       Sneha|   Mumbai| Mobile|electronics|32000|2024-01-05|  Completed|       37760.0|          High|
| ORD003|      C003|        Aman|Bangalore| Laptop|electronics|55000|2024-01-06|  Completed|       64900.0|          High|
| ORD004|      C004|       Pooja|    Delhi| Tablet|electronics|    0|2024-01-07|  Cancelled|           0.0|           Low|
| ORD006|      C006|       Rahul|   Mumbai| Mobile|electronics|    0|2024-01-08|  Completed|           0.0|           Low|
| ORD007|      C

In [43]:
from pyspark.sql.functions import year, month

In [44]:
order_year = price_category.withColumn("order_year",year(col("orderDate")))\
.withColumn("order_month",month(col("orderDate")))
order_year.show()

+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+--------------+----------+-----------+
|orderId|customerId|customerName|     city|product|   category|price| orderDate|orderStatus|price_with_tax|price_category|order_year|order_month|
+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+--------------+----------+-----------+
| ORD001|      C001|        Ravi|    Delhi| Laptop|electronics|45000|2024-01-05|  Completed|       53100.0|          High|      2024|          1|
| ORD002|      C002|       Sneha|   Mumbai| Mobile|electronics|32000|2024-01-05|  Completed|       37760.0|          High|      2024|          1|
| ORD003|      C003|        Aman|Bangalore| Laptop|electronics|55000|2024-01-06|  Completed|       64900.0|          High|      2024|          1|
| ORD004|      C004|       Pooja|    Delhi| Tablet|electronics|    0|2024-01-07|  Cancelled|           0.0|           Low|  

In [45]:
total_revenue_per_city = order_year.groupBy("city").agg({"price":"sum"})
total_revenue_per_city.show()

+---------+----------+
|     city|sum(price)|
+---------+----------+
|Bangalore|    203000|
|  Chennai|     83000|
|   Mumbai|    154000|
|    Delhi|    172000|
+---------+----------+



In [46]:
total_revenue_per_product = order_year.groupBy("product").agg({"price":"sum"})
total_revenue_per_product.show()

+-------+----------+
|product|sum(price)|
+-------+----------+
| Laptop|    370000|
| Mobile|    118000|
| Tablet|    124000|
+-------+----------+



In [50]:
total_revenue_per_city.orderBy("sum(price)",ascending=False).limit(3).show()

+---------+----------+
|     city|sum(price)|
+---------+----------+
|Bangalore|    203000|
|    Delhi|    172000|
|   Mumbai|    154000|
+---------+----------+



In [51]:
threshold = 30000
total_revenue_per_product = order_year.groupBy("product").agg({"price":"avg"})
total_revenue_per_product = total_revenue_per_product.filter(col("avg(price)") > threshold)
total_revenue_per_product.show()

+-------+------------------+
|product|        avg(price)|
+-------+------------------+
| Laptop|52857.142857142855|
+-------+------------------+



In [52]:
order_year.write.mode("overwrite").parquet("cleaned_orders.parquet")

In [54]:
check = spark.read.parquet("cleaned_orders.parquet")
check.printSchema()
check.show()

root
 |-- orderId: string (nullable = true)
 |-- customerId: string (nullable = true)
 |-- customerName: string (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- orderDate: date (nullable = true)
 |-- orderStatus: string (nullable = true)
 |-- price_with_tax: double (nullable = true)
 |-- price_category: string (nullable = true)
 |-- order_year: integer (nullable = true)
 |-- order_month: integer (nullable = true)

+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+--------------+----------+-----------+
|orderId|customerId|customerName|     city|product|   category|price| orderDate|orderStatus|price_with_tax|price_category|order_year|order_month|
+-------+----------+------------+---------+-------+-----------+-----+----------+-----------+--------------+--------------+----------+-----------+
| ORD001|      C0

In [56]:

order_year.write.mode("overwrite").orc("cleaned_orders.orc")

# order_year.write.mode("overwrite").format("avro").save("cleaned_orders.avro")

ORC (Optimized Row Columnar): A columnar storage format optimized for fast read performance in big data workloads, especially in Hive.

Avro: A row-oriented data serialization framework that uses JSON for defining data types and protocols, and serializes data in a compact binary format.


In [57]:
import os

initial_partitions = order_year.rdd.getNumPartitions()
print(f"Initial number of partitions for order_year: {initial_partitions}")

num_repartitions = 5
repartitioned_df = order_year.repartition(num_repartitions)
print(f"Number of partitions after repartitioning to {num_repartitions}: {repartitioned_df.rdd.getNumPartitions()}")

repartitioned_df.write.mode("overwrite").parquet("repartitioned_cleaned_orders.parquet")
repartitioned_df.write.mode("overwrite").orc("repartitioned_cleaned_orders.orc")




def count_data_files(path):
    try:
        files = [f for f in os.listdir(path) if not f.startswith('_') and not f.startswith('.')]
        if files and os.path.isdir(os.path.join(path, files[0])):
            count = 0
            for sub_dir in files:
                count += len([f for f in os.listdir(os.path.join(path, sub_dir)) if not f.startswith('_') and not f.startswith('.')])
            return count if count > 0 else len(files) # If data directly in main folder, return original count
        else:
            return len(files)
    except FileNotFoundError:
        return 0

parquet_file_count = count_data_files("repartitioned_cleaned_orders.parquet")
orc_file_count = count_data_files("repartitioned_cleaned_orders.orc")

print(f"\nFile counts for repartitioned data:")
print(f"  Parquet files: {parquet_file_count}")
print(f"  ORC files: {orc_file_count}")


print("\nExplain plan for the repartitioned DataFrame (detailed):")
repartitioned_df.explain(True)

Initial number of partitions for order_year: 1
Number of partitions after repartitioning to 5: 5

File counts for repartitioned data:
  Parquet files: 5
  ORC files: 5

Explain plan for the repartitioned DataFrame (detailed):
== Parsed Logical Plan ==
Repartition 5, true
+- Project [orderId#0, customerId#1, customerName#2, city#125, product#126, category#66, price#245, orderDate#37, orderStatus#8, price_with_tax#894, price_category#1183, order_year#1292, month(orderDate#37) AS order_month#1293]
   +- Project [orderId#0, customerId#1, customerName#2, city#125, product#126, category#66, price#245, orderDate#37, orderStatus#8, price_with_tax#894, price_category#1183, year(orderDate#37) AS order_year#1292]
      +- Project [orderId#0, customerId#1, customerName#2, city#125, product#126, category#66, price#245, orderDate#37, orderStatus#8, price_with_tax#894, CASE WHEN (price_with_tax#894 > cast(30000 as double)) THEN High WHEN ((price_with_tax#894 > cast(10000 as double)) AND (price_with_t